# JEM092 Asset Pricing - Homework 2
### Summer Semester 2021/2022

### Authors: *Petr Čala, Tereza Čechová, Vilém Krejcar*
___

First things first, we install and load the necessary packages. Then we arbitrarily set a seed for reproducibility.

In [12]:
# Required packages
packages <- c("stringr", "quantmod", "tseries", "dplyr", "xml2", "httr", "PortfolioAnalytics",
              "lubridate")

# Install packages not yet installed
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
  print(paste("Installing package ", packages[!installed_packages],"...", sep = ""))
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))
rm(list = ls()) #Clean environment
print('All packages loaded successfully...')

# Miscellaneous
options(repr.plot.width = 6, repr.plot.height = 5)
set.seed(420)

[1] "All packages loaded successfully..."


### Groundwork

Although we were able to obtain the data in the first homework, **we choose to use the provided dataset**, as this allows for easier and **more reliable reproducibility**. First we load the data, subset the necessary stocks, save these as new variables, and then discard the source data in order to alleviate some memory.

In [13]:
# Static variables
group_number <- 61505008
ticker_file <- "data/61505008_data_download.csv"
data_file <- "data/Asset_Pricing_HW_2_data.RData"
group_factor <- "data/third_factor_rand.csv"
data_files <- list(ticker_file, data_file, group_factor)

In [14]:
 # Set wd to project root
if (str_detect(getwd(), 'scripts')) {
    setwd('..')
}

# Assert presence of all required files
for (file in data_files) {
    if (!file.exists(file)) {
        print(paste0('File ', file, ' does not exist or has been misplaced.'))
    }
}

# Load tickers of required stocks
tickers <- as.character(unlist(read.csv(ticker_file)[2]))

# Check the assigned group factor
getFactor <- function(factor){
    data <- read.csv(factor)
    matching_row <- which(data[2] == group_number)
    assigned_factor <- as.character(data[matching_row, 3])
    print(paste0('The factor assigned to this group is ', assigned_factor, '.'))
}
getFactor(group_factor)

[1] "The factor assigned to this group is VOL."


While selecting only the desired tickers, we found that **a ticker that was assigned to our group was missing from the provided data**. We decided to remedy for this by writing a simple fix which replaces all of these missing assigned data by random data of a stock, that is provided in the source data set. As this was the case for only one stock, no trend bias should be introduced into the data.

In [83]:
# Load source data into the working directory
source_data <- load(data_file)

# Subset only for the stocks we will need
# Note - use these as BV$AAPL, not BV['AAPL']
BV <- book_value_sap500[tickers]
MKT <- MktCap_sap500[tickers]
OHLCV <- OHLCV_sap500[tickers]

# Assert presence of all assigned tickers in the source data - replace data of those that are missing
data_replacement_counter <- 1
for (ticker in tickers) {
    all_stock_names <- names(OHLCV)
    if (!ticker %in% all_stock_names) { # Missing file found
        print(paste0(ticker, ' is missing from the source data...'))
        missing_ticker_idx <- which(tickers == ticker) # Index of the missing file
        
        # Get replacement data
        while (names(book_value_sap500[data_replacement_counter]) %in% tickers) {
            data_replacement_counter <- data_replacement_counter + 1 # Search for a stock not from the assigned set
        }

        # Replace the data
        replacement_stock <- names(book_value_sap500)[data_replacement_counter] # Stock which shall serve as a replacement
        all_stock_names[missing_ticker_idx] <- replacement_stock # Fix names

        BV[missing_ticker_idx] <- book_value_sap500[replacement_stock]
        MKT[missing_ticker_idx] <- MktCap_sap500[replacement_stock]
        OHLCV[missing_ticker_idx] <- OHLCV_sap500[replacement_stock]

        names(BV) <- all_stock_names
        names(MKT) <- all_stock_names
        names(OHLCV) <- all_stock_names
        print(paste0('Data for ', ticker, ' replaced successfully with data from ', replacement_stock))

        data_replacement_counter <- data_replacement_counter + 1
    }
}

# Remove source data from working directory for better memory management
rm(list = source_data)

[1] "MOH is missing from the source data..."
[1] "Data for MOH replaced successfully with data from ABT"


### Data preprocessing

We define several functions to help us preprocess data.

In [84]:
######################################
### Data loading and preprocessing ###
######################################

# Download daily FF3 factors and return them as a single xts object
getDailyFF3 <- function() {
    # Load daily FF3 factors
    file_path = "data/F-F_Research_Data_Factors_daily.zip"

    if (!file.exists(file_path)) { # Download the file if not downloaded yet
    download.file(
        "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_TXT.zip",
        destfile = file_path 
    )
    }
    unzip("data/F-F_Research_Data_Factors_daily.zip", exdir = "data")

    ff3_factors <- read.delim(
        'data/F-F_Research_Data_Factors_daily.txt',
        col.names = c('t', 'mkt_rf', 'smb', 'hml', 'rf'),
        sep = '',
        nrows = 24957,
        header = FALSE,
        skip = 5,
        stringsAsFactors = FALSE
    )
    ff3_factors[['t']] <- as.Date(as.character(ff3_factors[['t']]), '%Y%m%d')
    ff3_factors <- as.xts(ff3_factors[, 2:5], order.by = ff3_factors[['t']])
    ff3_factors <- ff3_factors / 100
    return(ff3_factors)
}

# Input an xts object with daily data of market capitalization and return this as an xts data frame
#   with monthly market capitalization.
getMonthlyMarketCap <- function(daily_market_cap_data) {
    stock_names <- names(daily_market_cap_data)
    daily_market_caps <- do.call('cbind', daily_market_cap_data)
    monthly_market_caps <- apply.monthly(daily_market_caps, tail, 1)
    colnames(monthly_market_caps) <- stock_names
    index(monthly_market_caps) <- as.Date(as.yearmon(index(monthly_market_caps)), frac = 1)
    return(monthly_market_caps)
}

# Input a list of OHLCV datasets and return a similar object, only subsetted for adjusted prices. Discard the rest of the columns
getAdjustedPrices <- function(OHLCV_data) {
    if (!class(OHLCV_data) == 'list') {
        print('The input data must be a list')
        break
    }
    yankPrices <- function(data) {
        adj_col_idx <- which(str_detect(colnames(data), 'Adjusted')) # Get index of column containing adjusted prices
        prices <- data[, adj_col_idx]
        return(prices)
    }
    data_out <- lapply(OHLCV_data, yankPrices)
    data_out <- do.call('cbind', data_out)
    return(data_out)
}

# Input an xts data frame containing raw adjusted prices and convert these to daily or monthly returns
getReturns <- function(adjusted_prices_data, type = 'monthly') {
    if (!type %in% c('daily', 'monthly')) {
        stop('Can only handle daily or monthly returns calculation...')
    }
    getReturn <- function(data) {
        if (type == 'monthly') {
            return(suppressWarnings(monthlyReturn(data)))
        } else {
            return(suppressWarnings(dailyReturn(data)))
        }
    }    
    returns <- lapply(adjusted_prices_data, getReturn) # Calculate returns for all stocks
    returns <- do.call('cbind', returns)
    colnames(returns) <- gsub('Adjusted', 'returns', names(adjusted_prices_data)) # Rename columns
    return(returns)
}

#########################################
### Performance indicator calculation ###
#########################################

# Input daily and monthly returns data, along with daily ff3 factors, all as an xts object.
#   Return an xts object containing monthly betas for the input, calculated using last 12 months of data.
getMonthlyBetas <- function(daily_returns_data, monthly_returns_data, daily_ff3_data) {
    names(monthly_returns_data) <- gsub('returns', 'betas', names(monthly_returns_data))
    stocks <- names(monthly_returns_data) # Stock names - AAPL.betas
    monthly_dates <- index(monthly_returns_data) # Dates by months

    monthly_betas <- lapply(monthly_dates, function(end_date) { #Betas for all
        start_date <- end_date %m-% months(12) # Last 12 months of data
        date_range <- paste0(start_date, '/', end_date)
        daily_returns <- daily_returns_data[date_range]
        market_returns <- daily_ff3_data$mkt_rf[date_range]
        stocks_betas <- lapply(daily_returns, function(stock_returns) {
            model_data <- na.omit(cbind(stock_returns, market_returns))
            if (nrow(model_data) < 12 * 15) {
                return(NA)
            }
            model_betas <- lm(model_data[,1] ~ model_data[,2])
            return(model_betas$coefficients[[2]]) # Beta for given stock
        })
        return(unlist(stocks_betas)) # Betas for all stocks, given time frame
    })
    monthly_betas <- do.call('rbind', monthly_betas)
    monthly_betas <- as.xts(monthly_betas, order.by = monthly_dates)
    colnames(monthly_betas) <- stocks
    return(monthly_betas)
}

# Input a list of xts objects containing market capitalization info and convert these to monthly sizes
getMonthlySizes <- function(mkt_cap_data) {
    monthly_sizes <- lapply(mkt_cap_data, log) # Calculate monthly sizes
    monthly_sizes <- do.call('cbind', monthly_sizes)
    monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA))) # inf to NA
    colnames(monthly_sizes) <- paste0(colnames(monthly_sizes), '.sizes') # Rename columns
    return(monthly_sizes)
}

# Input two data frames (containing daily and monthly returns data) and return a data frame with monthly volatility
getMonthlyVolatilities <- function(daily_returns_data, monthly_returns_data) {
    names(monthly_returns_data) <- gsub('returns', 'volatility', names(monthly_returns_data))
    stocks <- names(monthly_returns_data) # Stock names - AAPL.returns
    monthly_dates <- index(monthly_returns_data) # Dates by months

    monthly_volatilities <- lapply(monthly_dates, function(end_date) { # Volatilities for all time frames, all stocks
        start_date <- end_date %m-% months(12) # Use last 12 months of data as suggested in the setup
        date_range <- paste0(start_date, '/', end_date)
        daily_returns <- daily_returns_data[date_range]
        stocks_volatilities <- lapply(daily_returns, function(stock) {
            average_stock_return <- mean(stock)
            nominator <- sum((stock - average_stock_return) ^ 2)
            denominator <- length(stock) - 1
            monthly_volatility <- 100 * sqrt(nominator/denominator) * sqrt(12)
            return(monthly_volatility) # Volatility for a given time frame, one stock
        })
        return(unlist(stocks_volatilities)) # Volatility for a given time frame, all stocks
    })
    monthly_volatilities <- do.call('rbind', monthly_volatilities) # Data frame of volatilities
    monthly_volatilities <- as.xts(monthly_volatilities, order.by = monthly_dates) # To xts
    colnames(monthly_volatilities) <- stocks
    return(monthly_volatilities)
}

In [85]:
# Compute the various metrics using previously defined functions
adjusted_prices <- getAdjustedPrices(OHLCV)
daily_returns <- getReturns(adjusted_prices, type = 'daily')
monthly_returns <- getReturns(adjusted_prices, type = 'monthly')
monthly_market_caps <- getMonthlyMarketCap(MKT)
daily_ff3 <- getDailyFF3()

In [211]:
# Compute the three desired factors (monthly)
monthly_sizes <- getMonthlySizes(monthly_market_caps) # 5s
monthly_betas <- getMonthlyBetas(daily_returns, monthly_returns, daily_ff3) # 2 mins
monthly_volatilities <- getMonthlyVolatilities(daily_returns, monthly_returns) # Monthly volatilities (vectors go brrrrrrr) - 9s

### Portfolio analysis

In [212]:
# Input monthly data to sort by, monthly returns data, and monthly market capital data.
#   Furthermore specify the number of quantiles and the method by which to compute the returns.
#   For this, two options are available - 'equal' (as in equally weighted) and 'value' (as in
#       value weighted).
getPortfolioReturns <- function(sort_variable_data, monthly_returns_data,
                                monthly_market_caps, n = 5, method = 'equal'){
    # Static
    diff_portfolio <- paste0(n, ' - 1')
    portfolio_colnames <- c(1:n, diff_portfolio)
    portfolio_returns_index <- index(sort_variable_data)[-1] # All but first element
    
    # Iterate over the months in the portfolio returns index and return return for each of them
    portfolio_returns <- lapply(portfolio_returns_index, function(current_month) {
        this_month_sort_variable <- sort_variable_data[current_month]
        not_na <- !is.na(this_month_sort_variable)
        if (sum(not_na) < 4*n) { # Not enough observations for this period
            return(c(rep(NA, n+1)))
        }

        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)
        if (!next_month %in% index(monthly_returns_data)) {
            return(c(rep(NA, n+1))) # Skip if no returns data in the next months
        }

        next_month_returns <- monthly_returns_data[next_month] # A vector for all stocks
        if (method == 'value') { # For value weighting
            next_month_market_caps <- monthly_market_caps[next_month]
            next_month_market_caps[is.na(next_month_market_caps)] <- 0 # No weights for missing obs
        }

        breakpoints <- quantile(this_month_sort_variable, prob = 0:n/n, na.rm = TRUE) # Quintiles

        # Compute portfolio returns
        this_period_returns <- c()
        for (i in breakpoints) { # Find all quintile breakpoints
            next_breakpoint_idx <- match(i, breakpoints) + 1
            next_breakpoint <- breakpoints[next_breakpoint_idx]
            if (!is.na(next_breakpoint)) { # While not out of quntiles
                filter <- (i < this_month_sort_variable) &
                          (this_month_sort_variable < next_breakpoint) &
                          (not_na)
                if (method == 'equal') {# Equally weighted portfolio
                    quintile_return <- mean(next_month_returns[,filter])
                } else if (method == 'value') {
                    quintile_return <- weighted.mean(t(next_month_returns[,filter]),
                                                    t(next_month_market_caps[,filter])) # Actual return
                } else {
                    stop('Incorrectly defined method. Select one of - equal, value.')
                }
                this_period_returns <- append(this_period_returns, quintile_return) # Store value 
            } else {
                break # Last iteration
            } 
        }
        diff_returns <- this_period_returns[[n]] - this_period_returns[[1]]
        this_period_returns <- append(this_period_returns, diff_returns) # Append returns of diff
        return(this_period_returns) # Returns for all quintiles for this period
    })
    portfolio_returns <- as.xts(do.call('rbind', portfolio_returns), # To xts
        order.by = as.Date(as.yearmon(portfolio_returns_index)))
    colnames(portfolio_returns) <- portfolio_colnames
    return(portfolio_returns)
}


In [213]:
portfolio_returns <- getPortfolioReturns(monthly_betas, monthly_returns, monthly_market_caps)

In [214]:
head(portfolio_returns['2015'])

                      1             2            3            4            5
2015-01-01           NA            NA           NA           NA           NA
2015-02-01  0.001974576  0.0030944735 -0.003114615 -0.008495491 -0.005803315
2015-03-01 -0.023752946 -0.0174280088 -0.006363546  0.002870140  0.021002123
2015-04-01           NA            NA           NA           NA           NA
2015-05-01 -0.035702142  0.0008197254 -0.009586905 -0.009241202 -0.024830853
2015-06-01  0.058954005  0.0330786286  0.033031995  0.005158851 -0.008878617
                  5 - 1
2015-01-01           NA
2015-02-01 -0.007777892
2015-03-01  0.044755069
2015-04-01           NA
2015-05-01  0.010871289
2015-06-01 -0.067832622